In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-14 13:48:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2024-07-14 13:48:58 (94.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
#number of unique characters in the text

chars = sorted(list(set(text)))
vocab_size= len(chars)
print(''.join(chars))
print(vocab_size)




 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# creating a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
# creating embedding for our model or encoder and decoder for out model
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [7]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# let's encode whole text
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:50]) # the first 50 characters

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56])


In [9]:
# split the data into test and validation
n = int(0.9*len(data)) # first 90% will be train, rest for validation
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

In [12]:
x,y

(tensor([18, 47, 56, 57, 58,  1, 15, 47]),
 tensor([47, 56, 57, 58,  1, 15, 47, 58]))

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")



when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [14]:
torch.manual_seed(1337)
# how many independent sequences will we process in parallel
batch_size = 4
# maximum context length for predictions
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

In [15]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [16]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {itos[target.item()]}")

when input is [24] the target: e
when input is [24, 43] the target: t
when input is [24, 43, 58] the target: '
when input is [24, 43, 58, 5] the target: s
when input is [24, 43, 58, 5, 57] the target:  
when input is [24, 43, 58, 5, 57, 1] the target: h
when input is [24, 43, 58, 5, 57, 1, 46] the target: e
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: a
when input is [44] the target: o
when input is [44, 53] the target: r
when input is [44, 53, 56] the target:  
when input is [44, 53, 56, 1] the target: t
when input is [44, 53, 56, 1, 58] the target: h
when input is [44, 53, 56, 1, 58, 46] the target: a
when input is [44, 53, 56, 1, 58, 46, 39] the target: t
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target:  
when input is [52] the target: t
when input is [52, 58] the target:  
when input is [52, 58, 1] the target: t
when input is [52, 58, 1, 58] the target: h
when input is [52, 58, 1, 58, 46] the target: a
when input is [52, 58, 1, 58, 46, 39] the target: t
when in

In [17]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx) # idx and targets are both
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self(idx)
      # focus only on the last time step
      logits = logits[:, -1, :] #
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # B, C
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) #
      # append to the sequence and continue
      idx = torch.cat((idx, idx_next), dim=1) #
    return idx

In [19]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [20]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [21]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [22]:
batch_size = 32
for steps in range(10000):
  # sample a batch of data
  xb, yb = get_batch('train')
  # evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item()) #have run the loop 3 times to get this loss


2.5727508068084717


In [23]:
# output after 3 iteration
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z


In [24]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a,1, keepdim = True)
b = torch.randint(0,10,(3,2)).float()

c = a@b
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [25]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [26]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] # (t,C)
    xbow[b,t] = torch.mean(xprev, 0)

In [27]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim = True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)


False

In [30]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim =-1)
xbow3= wei @x
torch.allclose(xbow, xbow3)

False

In [31]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# bulding single head attention block
head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias =False)
value = nn.Linear(C, head_size, bias = False)

k = key(x)
q = query(x)
wei = q @ k.transpose(-2,- 1)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [32]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [33]:
k = torch.randn(B,T, head_size)
q = torch.randn(B,T, head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5

In [34]:
k.var()

tensor(1.0449)

In [35]:
q.var()

tensor(1.0700)

In [36]:
wei.var()

tensor(1.0918)

In [38]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [39]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [42]:
class LayerNorm1d:
  def __init__(self, dim, eps=1e-5, momentum = 0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)


  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) #
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]



In [48]:
torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32,100)
x = module(x)
x.shape

torch.Size([32, 100])

In [49]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [50]:
x[0,:].mean(), x[0,:].std()

(tensor(-9.5367e-09), tensor(1.0000))

In [58]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 50000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [59]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [60]:
m = m.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.004225 M parameters
step 0: train loss 2.4550, val loss 2.4842
step 100: train loss 2.4533, val loss 2.4844
step 200: train loss 2.4540, val loss 2.4908
step 300: train loss 2.4527, val loss 2.4812
step 400: train loss 2.4495, val loss 2.4863
step 500: train loss 2.4515, val loss 2.4963
step 600: train loss 2.4593, val loss 2.4917
step 700: train loss 2.4511, val loss 2.4915
step 800: train loss 2.4541, val loss 2.4872
step 900: train loss 2.4484, val loss 2.4862
step 1000: train loss 2.4519, val loss 2.4939
step 1100: train loss 2.4470, val loss 2.4888
step 1200: train loss 2.4472, val loss 2.4830
step 1300: train loss 2.4542, val loss 2.4863
step 1400: train loss 2.4513, val loss 2.4829
step 1500: train loss 2.4552, val loss 2.4826
step 1600: train loss 2.4486, val loss 2.4873
step 1700: train loss 2.4477, val loss 2.4756
step 1800: train loss 2.4554, val loss 2.4789
step 1900: train loss 2.4551, val loss 2.4903
step 2000: train loss 2.4539, val loss 2.4874
step 2100: train loss 2.

In [61]:
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))



I menteyowfot wors t ochy ane st
AME:

Weso tht S:
A:
stho ek cctal the ceivek'e an, ncrd, achr thmo ld allearth wint y arice me trere,
And t; t th, orowidee posst's thend t illa ur,
inds ameno'dersutely ve d or o wex's whakes wain ul k ca fe,
UShilyotr y. wse shath d han morinel 'de, o m al

O:
Cun abe n t bir?
ABrin g
Busthatye atosous nd
Y:
Agr acen theicitito d mef was.

Tin ie h y plis hee.
Wisand g.

TIt d lad: hord ma lerileeen thentha-stie IENG, by.
FOVIICllors yon:
t ch'syo e iswiny Itedy m iter u.
Cly be and ber ak CENO:
YORod idefoowit oeneheghewh
NThernd-s, h w lotand the ace brt
Th thaicie
TEN knkenore his toarlitinithea; IUShe ch ofrd me mavets, s y oustitofo! Bicicefin pl s ay te meder stethe tearoo ialuly sta my fo thavesshedecy b h sesthou t st nt s EShi' f:
KELouin; Grizes bulf t.
NGorofaketh h, sthen y he it mathenche tr d. urnd d, ano thacan'se ice dain fidre,
HASTit INathe ive se IO:
Thtondu t Pe.
himy men t w ithes disu tavinof,
ANBousede,-avith
Ban'lllofo IINI n